In [ ]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torchvision
import torchvision.transforms as transforms
import tarfile
import pandas as pd
import os
import re
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from io import StringIO
from PIL import Image
import re
from sklearn.metrics import accuracy_score, f1_score, precision_score
import pickle
import torchvision.models as models

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class GoogLeNet(nn.Module):
    def __init__(self, n_classes, RGB_mean):
        super(GoogLeNet, self).__init__()
        self.RGB_mean = RBG_mean.to(device)
        self.n_classes = n_classes

        self.c1 = nn.Conv2d(3, 64, 7, stride =2, padding = 3)
        self.mp1 = nn.MaxPool2d(3, stride = 2, padding = 1)


        self.c21 = nn.Conv2d(64, 64, 1, stride=1, padding=0)
        self.c22 = nn.Conv2d(64, 192, 3, stride=1, padding=1)
        self.mp2 = nn.MaxPool2d(3, stride = 2)

        #inception layer (3a)
        self.c3a1 = nn.Conv2d(192, 64, 1, stride =1, padding = 0)
        
        self.c3a21 = nn.Conv2d(192, 96, 1, stride=1, padding =0)
        self.c3a22 = nn.Conv2d(96, 128, 3, stride=1, padding=1)
        
        self.c3a31 = nn.Conv2d(192, 16, 1 , stride=1, padding=0)
        self.c3a32 = nn.Conv2d(64, 32, 5, stride=1, padding=2)
        
        self.mp2a4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c3a4 = nn.Conv2d(192, 32, 1, stride=1, padding=0)

        #inception layer (3b)

        self.c3b1 = nn.Conv2d(256, 128, 1, stride =1, padding= 0)
        
        self.c3b21 = nn.Conv2d(256, 128, 1, stride=1, padding=0)
        self.c3b22 = nn.Conv2d(128, 192, 3, stride=1, padding=1)
        
        self.c3b31 = nn.Conv2d(256, 32, 1 , stride=1, padding=0)
        self.c3b32 = nn.Conv2d(32, 96, 5, stride=1, padding=2)
        
        self.mp2b4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c3b4 = nn.Conv2d(256, 64, 1, stride=1, padding=0)

        #max pool
        self.mp3 = nn.MaxPool2d(3, stride=2, padding=1)

        #inception layer (4a)
        self.c4a1 = nn.Conv2d(480, 192, 1, stride=1, padding=0)
        
        self.c4a21 = nn.Conv2d(480, 96, 1, stride=1 , padding=0)
        self.c4a22 = nn.Conv2d(96, 208, stride=1, padding=0)

        self.c4a31 = nn.Conv2d(480, 16, 1, stride=1, padding=0)
        self.c4a32 = nn.Conv2d(16, 48, 5, stride=1, padding=2)
        
        self.mp2a4 = nn.MaxPool2d(3, stride=1, padding=2)
        self.c4a42 = nn.Conv2d(480, 64, 1, stride=1, padding=0)

        #Auxilary 1
        self.apa1 = nn.AvgPool2d(5, stride = 3, padding = 0)
        self.ca1 = nn.Conv2d(512, 128 ,1, stride=1, padding=0)
        self.flat1 = nn.Flatten(-1,1)
        self.fca1 = nn.Linear(2048, 1024)
        self.a1drop = nn.Dropout(0.7)
        self.a1out = nn.Linear(1024, self.n_classes) 

        #Inception 4b
        self.c4b1 = nn.Conv2d(512, 160, 1, stride = 1, padding = 0)
        self.c4b21 = nn.Conv2d(512, 112, 1, stride = 1, padding = 0)
        self.c4b22 = nn.Conv2d(112, 224, 3, stride =1, padding = 0)
        self.c4b31 = nn.Conv2d(512, 24, 1, stride =1, padding = 0)
        self.c4b32 = nn.Conv2d(24, 64, 5, stride = 1, padding = 2)
        self.mp4b4 = nn.Conv2d(3, stride =1, padding=0)
        self.c4b42 = nn.Conv2d(512, 64, 1, stride=1, padding=0)

        #inception 4c
        self.c4c1 = nn.Conv2d(512, 128, 1, stride = 1, padding =0)
        self.c4c21 = nn.Conv2d(512, 128, 1, stride= 1, padding = 0)
        self.c4c22 = nn.Conv2d(128, 256, 3, stride =1, padding = 0)
        self.c4c31 = nn.Conv2d(512, 24, 1, stride=1, padding = 0)
        self.c4c32 = nn.Conv2d(24, 64, 5, stride =1, padding =2)
        self.mp4c4 = nn.Conv2d(3, stride=1, padding=0)
        self.c4c42 = nn.Conv2d(512, 64, 1, stride=1, padding =0)

        # Inception 4d
        self.c4d1 = nn.Conv2d(512, 112, 1, stride=1, padding=0)
        self.c4d21 = nn.Conv2d(512, 144, 1, stride=1, padding=0)
        self.c4d22 = nn.Conv2d(144, 288, 3, stride=1, padding=1)
        self.c4d31 = nn.Conv2d(512, 32, 1, stride=1, padding=0)
        self.c4d32 = nn.Conv2d(32, 64, 5, stride=1, padding=2)
        self.mp4d4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c4d4 = nn.Conv2d(512, 64, 1, stride=1, padding=0)

        #Auxiliary 2
        self.apa2 = nn.AvgPool2d(5, stride=3, padding=0)
        self.ca2 = nn.Conv2d(512,128,1, stride=1, padding=0)
        self.flat2 = nn.Flatten(-1,1)
        self.fca2 = nn.Linear(2028, 1024)
        self.a2drop = nn.Dropout(0.7)
        self.a2out = nn.Linear(1024,self.n_classes)

        # Inception 4e
        self.c4e1 = nn.Conv2d(528, 256, 1, stride=1, padding=0)
        self.c4e21 = nn.Conv2d(528, 160, 1, stride=1, padding=0)
        self.c4e22 = nn.Conv2d(160, 320, 3, stride=1, padding=1)
        self.c4e31 = nn.Conv2d(528, 32, 1, stride=1, padding=0)
        self.c4e32 = nn.Conv2d(32, 128, 5, stride=1, padding=2)
        self.mp4e4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c4e4 = nn.Conv2d(528, 128, 1, stride=1, padding=0)

        self.mp4 = nn.MaxPool2d(3, stride=2, padding=1)

        # Inception 5a
        self.c5a1 = nn.Conv2d(832, 256, 1, stride=1, padding=0)
        self.c5a21 = nn.Conv2d(832, 160, 1, stride=1, padding=0)
        self.c5a22 = nn.Conv2d(160, 320, 3, stride=1, padding=1)
        self.c5a31 = nn.Conv2d(832, 32, 1, stride=1, padding=0)
        self.c5a32 = nn.Conv2d(32, 128, 5, stride=1, padding=2)
        self.mp5a4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c5a4 = nn.Conv2d(832, 128, 1, stride=1, padding=0)

         # Inception 5b
        self.c5b1 = nn.Conv2d(832, 384, 1, stride=1, padding=0)
        self.c5b21 = nn.Conv2d(832, 192, 1, stride=1, padding=0)
        self.c5b22 = nn.Conv2d(192, 384, 3, stride=1, padding=1)
        self.c5b31 = nn.Conv2d(832, 48, 1, stride=1, padding=0)
        self.c5b32 = nn.Conv2d(48, 128, 3, stride=1, padding=1)
        self.mp5b4 = nn.MaxPool2d(3, stride=1, padding=1)
        self.c5b4 = nn.Conv2d(832, 128, 1, stride=1, padding=0)

        self.ap = nn.AvgPool2d(7, stride =1)
        self.drop = nn.Dropout(0.4)
        self.flat = nn.Flatten(1,-1)
        self.out = nn.Linear(1024, self.n_classes)
    
    def forward(self, x, auxiliary=True):

        x = x - self.RGB_mean[None, :, None, None]

        # Layer 1
        x = self.mp1(F.relu(self.c1(x)))

        # Layer 2
        x = self.mp2(F.relu(self.c22(F.relu(self.c21(x)))))

        # Layer 3a
        x1 = F.relu(self.c3a1(x))
        x2 = F.relu(self.c3a22(F.relu(self.c3a21(x))))
        x3 = F.relu(self.c3a32(F.relu(self.c3a31(x))))
        x4 = F.relu(self.c3a4(self.mp3a4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Layer 3b
        x1 = F.relu(self.c3b1(x))
        x2 = F.relu(self.c3b22(F.relu(self.c3b21(x))))
        x3 = F.relu(self.c3b32(F.relu(self.c3b31(x))))
        x4 = F.relu(self.c3b4(self.mp3b4(x)))
        x = self.mp3(torch.cat((x1, x2, x3, x4), 1))

        # Layer 4a
        x1 = F.relu(self.c4a1(x))
        x2 = F.relu(self.c4a22(F.relu(self.c4a21(x))))
        x3 = F.relu(self.c4a32(F.relu(self.c4a31(x))))
        x4 = F.relu(self.c4a4(self.mp4a4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Auxiliary 1
        if auxiliary == True:
            z1 = self.flat1(F.relu(self.ca1(self.apa1(x))))
            z1 = self.a1out(self.a1drop(F.relu(self.fca1(z1))))
        else:
            z1 = None

        # Layer 4b
        x1 = F.relu(self.c4b1(x))
        x2 = F.relu(self.c4b22(F.relu(self.c4b21(x))))
        x3 = F.relu(self.c4b32(F.relu(self.c4b31(x))))
        x4 = F.relu(self.c4b4(self.mp4b4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Layer 4c
        x1 = F.relu(self.c4c1(x))
        x2 = F.relu(self.c4c22(F.relu(self.c4c21(x))))
        x3 = F.relu(self.c4c32(F.relu(self.c4c31(x))))
        x4 = F.relu(self.c4c4(self.mp4c4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Layer 4d
        x1 = F.relu(self.c4d1(x))
        x2 = F.relu(self.c4d22(F.relu(self.c4d21(x))))
        x3 = F.relu(self.c4d32(F.relu(self.c4d31(x))))
        x4 = F.relu(self.c4d4(self.mp4d4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Auxiliary 2
        if auxiliary == True:
            z2 = self.flat(F.relu(self.ca2(self.apa2(x))))
            z2 = self.a2out(self.a2drop(F.relu(self.fca2(z2))))
        else:
            z2 = None

        # Layer 4e
        x1 = F.relu(self.c4e1(x))
        x2 = F.relu(self.c4e22(F.relu(self.c4e21(x))))
        x3 = F.relu(self.c4e32(F.relu(self.c4e31(x))))
        x4 = F.relu(self.c4e4(self.mp4e4(x)))
        x = self.mp4(torch.cat((x1, x2, x3, x4), 1))

        # Layer 5a
        x1 = F.relu(self.c5a1(x))
        x2 = F.relu(self.c5a22(F.relu(self.c5a21(x))))
        x3 = F.relu(self.c5a32(F.relu(self.c5a31(x))))
        x4 = F.relu(self.c5a4(self.mp5a4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Layer 5b
        x1 = F.relu(self.c5b1(x))
        x2 = F.relu(self.c5b22(F.relu(self.c5b21(x))))
        x3 = F.relu(self.c5b32(F.relu(self.c5b31(x))))
        x4 = F.relu(self.c5b4(self.mp5b4(x)))
        x = torch.cat((x1, x2, x3, x4), 1)

        # Final Output
        x = self.out(self.flat(self.drop(self.ap(x))))

        return x, z1, z2

    def predict(self, x):

        y_hat = self.forward(x)
        y_hat = torch.argmax(y_hat, axis=1)
        return y_hat

googlenet = pickle.load(open('/content/drive/My Drive/google_init.sav', 'rb'))
classifier = GoogLeNet(7, RGB_mean)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)

params = list(googlenet.parameters())

with torch.no_grad():
    
    classifier.c1.weight = params[0]
    # classifier.c1.bias = params[1]

    classifier.c21.weight = params[3]
    # classifier.c21.bias = params[4]
    classifier.c22.weight = params[6]
    # classifier.c22.bias = params[7]

    classifier.c3a1.weight = params[9]
    # classifier.c3a1.bias = params[10]
    classifier.c3a21.weight = params[12]
    # classifier.c3a21.bias = params[13]
    classifier.c3a22.weight = params[15]
    # classifier.c3a22.bias = params[16]
    classifier.c3a31.weight = params[18]
    # classifier.c3a31.bias = params[19]
    classifier.c3a32.weight = params[21]
    # classifier.c3a32.bias = params[22]
    classifier.c3a4.weight = params[24]
    # classifier.c3a4.bias = params[25]
    
    classifier.c3b1.weight = params[27]
    # classifier.c3b1.bias = params[28]
    classifier.c3b21.weight = params[30]
    # classifier.c3b21.bias = params[31]
    classifier.c3b22.weight = params[33]
    # classifier.c3b22.bias = params[34]
    classifier.c3b31.weight = params[36]
    # classifier.c3b31.bias = params[37]
    classifier.c3b32.weight = params[39]
    # classifier.c3b32.bias = params[40]
    classifier.c3b4.weight = params[42]
    # classifier.c3b4.bias = params[43]
    
    classifier.c4a1.weight = params[45]
    # classifier.c4a1.bias = params[46]
    classifier.c4a21.weight = params[48]
    # classifier.c4a21.bias = params[49]
    classifier.c4a22.weight = params[51]
    # classifier.c4a22.bias = params[52]
    classifier.c4a31.weight = params[54]
    # classifier.c4a31.bias = params[55]
    classifier.c4a32.weight = params[57]
    # classifier.c4a32.bias = params[58]
    classifier.c4a4.weight = params[60]
    # classifier.c4a4.bias = params[61]

    classifier.c4b1.weight = params[63]
    # classifier.c4b1.bias = params[64]
    classifier.c4b21.weight = params[66]
    # classifier.c4b21.bias = params[67]
    classifier.c4b22.weight = params[69]
    # classifier.c4b22.bias = params[70]
    classifier.c4b31.weight = params[72]
    # classifier.c4b31.bias = params[73]
    classifier.c4b32.weight = params[75]
    # classifier.c4b32.bias = params[76]
    classifier.c4b4.weight = params[78]
    # classifier.c4b4.bias = params[79]

    classifier.c4c1.weight = params[81]
    # classifier.c4c1.bias = params[82]
    classifier.c4c21.weight = params[84]
    # classifier.c4c21.bias = params[85]
    classifier.c4c22.weight = params[87]
    # classifier.c4c22.bias = params[88]
    classifier.c4c31.weight = params[90]
    # classifier.c4c31.bias = params[91]
    classifier.c4c32.weight = params[93]
    # classifier.c4c32.bias = params[94]
    classifier.c4c4.weight = params[96]
    # classifier.c4c4.bias = params[97]

    classifier.c4d1.weight = params[99]
    # classifier.c4d1.bias = params[100]
    classifier.c4d21.weight = params[102]
    # classifier.c4d21.bias = params[103]
    classifier.c4d22.weight = params[105]
    # classifier.c4d22.bias = params[106]
    classifier.c4d31.weight = params[108]
    # classifier.c4d31.bias = params[109]
    classifier.c4d32.weight = params[111]
    # classifier.c4d32.bias = params[112]
    classifier.c4d4.weight = params[114]
    # classifier.c4d4.bias = params[115]

    classifier.c4e1.weight = params[117]
    # classifier.c4e1.bias = params[118]
    classifier.c4e21.weight = params[120]
    # classifier.c4e21.bias = params[121]
    classifier.c4e22.weight = params[123]
    # classifier.c4e22.bias = params[124]
    classifier.c4e31.weight = params[126]
    # classifier.c4e31.bias = params[127]
    classifier.c4e32.weight = params[129]
    # classifier.c4e32.bias = params[130]
    classifier.c4e4.weight = params[132]
    # classifier.c4e4.bias = params[133]

    classifier.c5a1.weight = params[135]
    # classifier.c5a1.bias = params[136]
    classifier.c5a21.weight = params[138]
    # classifier.c5a21.bias = params[139]
    classifier.c5a22.weight = params[141]
    # classifier.c5a22.bias = params[142]
    classifier.c5a31.weight = params[144]
    # classifier.c5a31.bias = params[145]
    classifier.c5a32.weight = params[147]
    # classifier.c5a32.bias = params[148]
    classifier.c5a4.weight = params[150]
    # classifier.c5a4.bias = params[151]

    classifier.c5b1.weight = params[153]
    # classifier.c5b1.bias = params[154]
    classifier.c5b21.weight = params[156]
    # classifier.c5b21.bias = params[157]
    classifier.c5b22.weight = params[159]
    # classifier.c5b22.bias = params[160]
    classifier.c5b31.weight = params[162]
    # classifier.c5b31.bias = params[163]
    classifier.c5b32.weight = params[165]
    # classifier.c5b32.bias = params[166]
    classifier.c5b4.weight = params[168]
    # classifier.c5b4.bias = params[169]

classifier = classifier.to(device)

old_loss = np.inf

max_epoch = 500
losses = []
for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat, y_hat1, y_hat2 = classifier(X)
        
        # Calculate Loss (Cross Entropy)
        loss_main = criterion(y_hat, y)
        loss1 = criterion(y_hat1, y)
        loss2 = criterion(y_hat2, y)

        # Weighted Loss
        loss = loss_main + 0.3*loss1 + 0.3*loss2

        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()*len(X)/train_size
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    losses.append(running_loss)
    if abs(running_loss-old_loss)/running_loss < 0.05 and running_loss<0.2:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')
plt.plot(losses)
plt.ylabel('Loss')
plt.xlabel('Iter Number')
plt.title('Convergence monitor plot')
plt.show()

with torch.no_grad():
    
    train_loss = 0.0
    y_train = []
    y_train_pred = []

    for data in trainloader:

        X, y = data[0].to(device), data[1].to(device)
        y_hat, _, _ = classifier(X)      
        train_loss += criterion(y_hat, y)*len(X)/train_size
        
        y_train.extend(list(y.detach().cpu().numpy()))
        y_train_pred.extend(list(torch.argmax(y_hat, axis=1).detach().cpu().numpy()))

print('Train Loss =', train_loss.item())
pd.DataFrame(confusion_matrix(y_train, y_train_pred))

'''
Test Loss = 1.312967300415039
0	1	2	3	4	5	6
0	4	0	0	0	1	1	2
1	0	4	0	1	0	1	2
2	1	0	2	0	0	0	0
3	0	0	1	8	0	1	0
4	0	0	0	0	4	0	1
5	0	0	0	0	1	2	1
6	0	0	1	0	0	0	3
'''
acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')

print('Test Accuracy =', acc, 'Test Precision =', prec, 'Test F1 =', f1)
torch.save(classifier, '/content/drive/My Drive/incept_solution_model.pt')

